In [1]:
# import dill
import pickle

# Сбор данных из vk

In [2]:
import os
import zipfile
import pickle
# из 1 дз
class FileStorage():

    def __init__(self, file_name):
        self.file_name = file_name
        self.user_ids = []

    def read_data(self):
        if not os.path.exists(self.file_name):
            raise StopIteration

        data = {}
        with zipfile.ZipFile(self.file_name) as zf:
            self.user_ids = zf.namelist()
            for file in self.user_ids:
                data[file] = pickle.loads(zf.read(file))
        return data

    def write_data(self, data):
        """
        :param data_array: dict content user name and user data
        """
        with zipfile.ZipFile(self.file_name, mode='w', compression=zipfile.ZIP_DEFLATED) as zf:
            for key, value in data.items():
                zf.writestr(key, pickle.dumps(value))

    def append_data(self, data):
        """
        :param data: dict content user name and user data
        """
        old_data = self.read_data()
        old_data.update(data)
        self.write_data(old_data)


In [3]:
import logging

FORMAT = '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
LOG_FILE = 'vk_user_analise.log'

log_formatter = logging.Formatter(FORMAT)
logger = logging.getLogger('vk_user_analise')
logger.setLevel(logging.INFO)

fh = logging.FileHandler(LOG_FILE)
fh.setFormatter(log_formatter)
logger.addHandler(fh)

ch = logging.StreamHandler()
ch.setFormatter(log_formatter)
logger.addHandler(ch)

In [4]:
import logging
from time import sleep,time
import numpy as np         
import vk

loggerS = logging.getLogger('vk_user_analise.scrapper')

class Scrapper(object):
    def __init__(self, delay=1):
        self.user_ids_file = 'users.txt'
        self.delay = delay
        
    def _get_user_ids(self):
        all_users = np.arange(1890000,dtype=np.int)
        np.random.shuffle(all_users)
        user_ids=all_users[:100000]
        
        with open(self.user_ids_file, 'w') as output_file:
            output_file.write(';'.join(map(str, user_ids)))
            
        loggerS.info('total: {} users'.format(len(user_ids)))
        
    def _wait(self, c, s):
        count = c+1
        start = s
        if count >=2:
            count = 0
            tm = time()-start
            if tm <= 1:
                sleep(1)
            start=time()
        return count, start
    
    def scrap_process(self, storage, update=False):
        loggerS.info('Start scrapping')

        if not os.path.exists(self.user_ids_file):
            self._get_user_ids()
        with open(self.user_ids_file, 'r') as f:
            user_ids = f.read().split(';')
        user_total = len(user_ids)

        data = {}
        user_done = []
        if not os.path.exists(storage.file_name) or update:
            storage.write_data(data)
        else:
            storage.read_data()
            user_done = storage.user_ids
        
        login = 'obi-wan.kenobi@list.ru'
#         login='MyLogin'
        password = 'GmFA2yPdzl.d'
#         password = 'MyPass'
        vk_id = '6815038'
#         vk_id = 'MyAppId'
        api_version = '5.92'

        session = vk.AuthSession(app_id=vk_id, user_login=login, user_password=password) 
        api = vk.API(session)
        fields = ['bdate','country','city','counters','education','career','occupation','personal',
                  'relation','sex','about','activities','books','games','interests','movies',
                  'music','personal','quotes','tv']
            
        start = time()
        count = 0
        last_percent = 0
        for i,user_id in enumerate(user_ids):
            try:
                if user_id in user_done:
                    continue
                    
                percent = int(i / user_total * 100)
                user_data = api.users.get(user_ids=user_id, v=api_version, fields=fields)
                count, start = self._wait(count,start)
#                 print(user_id)
#                 print(user_data[0])
                try:
                    if not user_data[0]['is_closed']:
                        user_data[0]['groups'] = api.users.getSubscriptions(user_id=user_id,extended=1,count=200, v=api_version)
                        count, start = self._wait(count,start)
                except KeyError:
                    pass
                data[user_id] = user_data[0]
                
                if len(data)>=100:
                    storage.append_data(data)
                    data={}
                    
                if percent != last_percent:
                    loggerS.info(f'{percent}% done')
                    last_percent = percent
                
                user_done.append(user_id)
                
            except KeyboardInterrupt:
                loggerS.info('Scrapping interrupt by keyboard')
                break
        
        storage.append_data(data)
        
        loggerS.info('Scrapping done')
    

In [5]:
import re
import numpy as np
import pandas as pd

class FeatureExtractor():
    def __init__(self, year):
        self.fields = ['id','first_name','last_name','sex','age','country','city',
                       'videos','audios','photos','friends','groups','university',
                       'activities','books','games','interests','movies','music',
                       'personal','quotes','tv']
        self.count_open = 0
        self.count_closed = 0
        self.count_deleted = 0
        self.year = year
        
    def _get_age(self, data):
        age = np.NaN
        try:
            match = re.search(r'(\d{4})',data['bdate'])
            if match:
                age = int(self.year - int(match.group(1)))
        except KeyError:
            pass
        return age
    
    def _get_sex(self,data):
        return data['sex']
    
    def _get_country(self,data):
        country = np.NaN
        try:
            country = data['country']['title']
            if country == '':
                country = np.NaN
        except KeyError:
            pass
        return country
    
    def _get_city(self,data):
        city = np.NaN
        try:
            city = data['city']['title']
            if city == '':
                city = np.NaN
        except KeyError:
            pass
        return city
    
    def _get_videos(self,data):
        videos = np.NaN
        try:
            videos = data['counters']['videos']
            if videos == '':
                videos = np.NaN
        except KeyError:
            pass
        return videos
    
    def _get_audios(self,data):
        audios=np.NaN
        try:
            audios = data['counters']['audios']
            if audios == '':
                audios = np.NaN
        except KeyError:
            pass
        return audios
    
    def _get_photos(self,data):
        photos=np.NaN
        try:
            photos = data['counters']['photos']
            if photos == '':
                photos = np.NaN
        except KeyError:
            pass
        return photos
    
    def _get_university(self,data):
        university=np.NaN
        try:
            university = data['university_name']
            if university == '':
                university = np.NaN
        except KeyError:
            pass
        return university
    
    def _get_activities(self,data):
        activities=np.NaN
        try:
            activities = data['activities']
            if activities == '':
                activities = np.NaN
        except KeyError:
            pass
        return activities
    
    def _get_books(self,data):
        books=np.NaN
        try:
            books = data['books']
            if books == '':
                books = np.NaN
        except KeyError:
            pass
        return books
    
    def _get_games(self,data):
        games=np.NaN
        try:
            games = data['games']
            if games == '':
                games = np.NaN
        except KeyError:
            pass
        return games
    
    def _get_movies(self,data):
        movies=np.NaN
        try:
            movies = data['movies']
            if movies == '':
                movies = np.NaN
        except KeyError:
            pass
        return movies
    
    def _get_personal(self,data):
        personal=np.NaN
        try:
            personal = data['personal']
            if personal == '':
                personal = np.NaN
        except KeyError:
            pass
        return personal
    
    def _get_quotes(self,data):
        quotes=np.NaN
        try:
            quotes = data['quotes']
            if quotes == '':
                quotes = np.NaN
        except KeyError:
            pass
        return quotes
    
    def _get_tv(self,data):
        tv=np.NaN
        try:
            tv = data['tv']
            if tv == '':
                tv = np.NaN
        except KeyError:
            pass
        return tv
    
    def _get_groups(self,data):
        groups=np.NaN
        try:
            groups = ','.join([gr['name'] for gr in data['groups']['items'] if gr['type']=='page'])
            groups=groups.replace('\t',' ')
            groups=groups.replace('\n','')
            groups=groups.replace('\r','')
            groups=groups.replace(', ,',',')
            if groups == '':
                groups = np.NaN
        except KeyError:
            pass
        return groups
    
    def _get_music(self,data):
        music=np.NaN
        try:
            music = data['music']
            if music == '':
                music = np.NaN
        except KeyError:
            pass
        return music
    
    def _get_friends(self,data):
        friends=np.NaN
        try:
            friends = data['counters']['friends']
            if friends == '':
                friends = np.NaN
        except KeyError:
            pass
        return friends
    
    def _get_interests(self,data):
        interests=np.NaN
        try:
            interests = data['interests']
            if interests == '':
                interests = np.NaN
        except KeyError:
            pass
        return interests
        
    def transform(self, data):
        df_data = {field:[] for field in self.fields}
#         count = 0
#         max_count = 10
        for user_id, user_data in data.items():
#             count+=1
#             if count>max_count:
#                 break

            try:
                if user_data['is_closed']:
                    self.count_closed+=1
                    continue
            except KeyError:
                self.count_deleted+=1
                continue
            self.count_open+=1
            # обязательные поля
            df_data['id'].append(user_id)
            df_data['first_name'].append(user_data['first_name'])
            df_data['last_name'].append(user_data['last_name'])
            # необязательные
            df_data['sex'].append(self._get_sex(user_data))
            df_data['age'].append(self._get_age(user_data))
            df_data['country'].append(self._get_country(user_data))
            df_data['city'].append(self._get_city(user_data))
            df_data['videos'].append(self._get_videos(user_data))
            df_data['audios'].append(self._get_audios(user_data))
            df_data['photos'].append(self._get_photos(user_data))
            df_data['friends'].append(self._get_friends(user_data))
            df_data['groups'].append(self._get_groups(user_data))
            df_data['university'].append(self._get_university(user_data))
            df_data['activities'].append(self._get_activities(user_data))
            df_data['books'].append(self._get_books(user_data))
            df_data['games'].append(self._get_games(user_data))
            df_data['movies'].append(self._get_movies(user_data))
            df_data['music'].append(self._get_music(user_data))
            df_data['personal'].append(self._get_personal(user_data))
            df_data['quotes'].append(self._get_quotes(user_data))
            df_data['tv'].append(self._get_tv(user_data))
            df_data['interests'].append(self._get_interests(user_data))
        return pd.DataFrame(df_data)


In [6]:
update_data=False
collect_data=False
# collect_data=True

In [7]:
data_file = 'test.zip'
fs = FileStorage(data_file)
if collect_data:
    sc = Scrapper()
    sc.scrap_process(fs,update_data)
# fs.read_data()

# Анализ данных

In [8]:
# collect_data=True
collect_data=False
if collect_data:
    fe = FeatureExtractor(2018)
    df = fe.transform(fs.read_data())
    print(fe.count_open, fe.count_closed, fe.count_deleted)
    df.to_csv("data.csv", index=False, sep='\t')
else:
    df = pd.read_csv("data.csv", sep='\t',dtype={'id': np.int,'sex':np.int})
#     df['age'] = pd.to_numeric(df['age'], errors='coerce')
#     df['age'] = df['age'].fillna(0).astype(np.int)
    
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65569 entries, 0 to 65568
Data columns (total 22 columns):
id            65569 non-null int64
first_name    65568 non-null object
last_name     65567 non-null object
sex           65569 non-null int64
age           26585 non-null float64
country       61093 non-null object
city          57448 non-null object
videos        65569 non-null int64
audios        65569 non-null int64
photos        65569 non-null int64
friends       65569 non-null int64
groups        52392 non-null object
university    15397 non-null object
activities    5147 non-null object
books         5376 non-null object
games         4466 non-null object
interests     6342 non-null object
movies        5807 non-null object
music         5961 non-null object
personal      12885 non-null object
quotes        6505 non-null object
tv            3971 non-null object
dtypes: float64(1), int64(6), object(15)
memory usage: 11.0+ MB


In [9]:
import pandas_profiling

pandas_profiling.ProfileReport(df)

Number of variables,22
Number of observations,65569
Total Missing (%),45.0%
Total size in memory,11.0 MiB
Average record size in memory,176.0 B
Numeric,6
Categorical,15
Boolean,1
Date,0
Text (Unique),0
Rejected,0


Возраст распределен в пределах 25-50 лет в основном, остальные или скрыты, или завышены. Поэтому предсказывать можно только в этом диапозоне.
В городах много уникальных названий, но если предположить, что преимущественно живут в России в крупных городах. Те кто скрывают свое местоположение, окажутся в шуме. Поэтому можно попробовать предсказать город по группам в которых состоит человек.

# Предсказание города в котором живет человек по группам в которых состоит


Логично вытаскивать названия из групп. Но далеко не у всех пользователей в группах есть названия городов. Так же группы вк представляют интересы основных групп пользователей в конкретных городах. Я бы еще и признак возраста добавил бы, но он в маленьком диапазоне. Для интереса вообще бы названия городов исключил

Разобьем группы и на обучающую и тестовую, на тестовой проверим после нахождения параметров модели.

In [10]:
try:
    with open('df_train.pkl', 'rb') as f:
        df_train = pickle.load(f)
    with open('df_test.pkl', 'rb') as f:
        df_test = pickle.load(f)
except FileNotFoundError:
    l = int(df.shape[0]/2)
    df_train = df[:l]
    df_test = df[l:]
    with open('df_train.pkl', 'wb') as f:
        pickle.dump(df_train, f)
    with open('df_test.pkl', 'wb') as f:
        pickle.dump(df_test, f)
del df
print(df_train.shape)
print(df_test.shape)


(32784, 22)
(32785, 22)


In [11]:
# import nltk
# nltk.download('punkt')
# nltk.download("stopwords")

In [61]:
from sklearn.feature_extraction.text import TfidfVectorizer
# pip install git+git://github.com/scikit-learn/scikit-learn.git
from sklearn.preprocessing import LabelEncoder
from nltk.corpus import stopwords
from nltk.stem.snowball import RussianStemmer
import nltk
import string


def my_tokenizer(text):
    tokens = nltk.word_tokenize(text)
    stems = []
    for item in tokens:
        stems.append(RussianStemmer().stem(item))
    return stems

class Get_xy():
    def __init__(self,n_features=200):
        self.le = None
        self.vectorizer = None
        self.delete_str = []
        self.n_features = n_features
    
    def fit(self, df):
        df_xy = df.copy()
        
        self.le = LabelEncoder()
#         self.le = CategoricalEncoder(handle_unknown='ignore') 
        self.le.fit(df_xy['city'])
        
        stop_words = stopwords.words('russian')
        stop_words.extend(stopwords.words('english'))
        stop_words.extend(['©', '«', '»','—', '•', '№', '●', '★', '♥', 'ツ'])
        stop_words = [my_tokenizer(sw)[0] for sw in stop_words]
        self.delete_str = string.punctuation + '1234567890'
        df_xy['groups_clean'] = [s.lower().translate(str.maketrans(self.delete_str, ' '*len(self.delete_str))) 
                             for s in df_xy['groups']]
        
        self.vectorizer = TfidfVectorizer(max_df=0.95, min_df=2,
                                 max_features=self.n_features,
                                 stop_words=stop_words,
                                 tokenizer=my_tokenizer)
        self.vectorizer.fit(df_xy['groups_clean'])
        
        
    def transform(self, df):
        df_xy = df.copy()
        df_xy['groups_clean'] = [s.lower().translate(str.maketrans(self.delete_str, ' '*len(self.delete_str))) 
                             for s in df_xy['groups']]
        y = self.le.transform(df_xy['city'])
        X = self.vectorizer.transform(df_xy['groups_clean'])
        
        return X,y
    
    def fit_transform(self, df):
        self.fit(df)
        return self.transform(df)

try:
    with open('xy_train.pkl', 'rb') as f:
        X = pickle.load(f)
        y = pickle.load(f)
        get_xy = pickle.load(f)
except FileNotFoundError:
    df_xy = df_train.dropna(subset=['city', 'groups'])
    get_xy = Get_xy()
    X,y = get_xy.fit_transform(df_xy)
    del df_xy
    with open('xy_train.pkl', 'wb') as f:
        pickle.dump(X, f)
        pickle.dump(y, f)
        pickle.dump(get_xy, f)
print(get_xy.vectorizer.get_feature_names())

['aliexpress', 'appét', 'art', 'bon', 'club', 'cook', 'decor', 'design', 'e', 'english', 'fashion', 'fitness', 'gif', 'handmad', 'hd', 'histor', 'kudag', 'l', 'lif', 'liv', 'music', 'onlin', 'ru', 'scienc', 'u', 'vide', 'world', 'авт', 'академ', 'английск', 'анекдот', 'арт', 'аудиокниг', 'афиш', 'барахолк', 'бесплатн', 'бизнес', 'блог', 'ваш', 'вдохновен', 'велик', 'виде', 'вконтакт', 'вкусн', 'вопрос', 'вязан', 'г', 'город', 'горя', 'групп', 'дар', 'дач', 'девушк', 'декор', 'ден', 'дет', 'детск', 'дизайн', 'дневник', 'добр', 'дом', 'домашн', 'дтп', 'душ', 'женск', 'женщин', 'жизн', 'журна', 'здоров', 'игр', 'ид', 'интеллектуальн', 'интересн', 'интернет', 'интерьер', 'искусств', 'истор', 'ищ', 'йог', 'кадр', 'кажд', 'квартир', 'квартирн', 'кин', 'киноман', 'клуб', 'книг', 'кот', 'красот', 'кулинар', 'кулинарн', 'кухн', 'лайфхак', 'лепр', 'литератур', 'любов', 'люд', 'магазин', 'макияж', 'мам', 'маникюр', 'мастер', 'мастерск', 'мир', 'мод', 'модел', 'москв', 'мотивац', 'мужск', 'музык',

In [13]:
# dill.dump_session('hw_vector.db')

In [14]:
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV, cross_val_score
from scipy.stats import randint,expon
from time import time
from sklearn.metrics import mean_squared_error, make_scorer, accuracy_score
from sklearn.model_selection import train_test_split

def report(search_res):
    print("Best param: {0}".format(search_res.best_params_))
    print("Best score: {0}".format(search_res.best_score_))

def test_model(model,X,y):
    X_train, X_test, y_train, y_test = train_test_split(X, y)

    model.fit(X_train, y_train)
    print("score:",accuracy_score(model.predict(X_test), y_test))

def gs_model(model,param,X,y,n_iter_search = 20, cv=3, jobs=4, verbose=10):

#     rmse_scorer = make_scorer(mean_squared_error)
#     search = RandomizedSearchCV(model, param_distributions=param, scoring=rmse_scorer,
#                                    n_iter=n_iter_search, cv=cv, n_jobs=4)
    search = RandomizedSearchCV(model, param_distributions=param, scoring="accuracy",
                                   n_iter=n_iter_search, cv=cv, n_jobs=jobs, verbose=verbose, pre_dispatch=4)
#     search = RandomizedSearchCV(model, param_distributions=param, scoring="neg_mean_squared_error",
#                                    n_iter=n_iter_search, cv=cv, verbose=verbose)
#     search = GridSearchCV(model, param, scoring="neg_mean_squared_error", cv=cv, n_jobs=4, verbose=10)
    start=time()
    search.fit(X, y)
    print("RandomizedSearchCV took %.2f seconds for %d candidates parameter settings." % ((time() - start), n_iter_search))
#     report(search)
    return search

def cv_model(model,X,y,jobs=4):
    scores = cross_val_score(model, X, y, cv=5, scoring='accuracy',n_jobs=jobs)
    print(scores)
    print(scores.mean(),scores.std())

In [15]:
search_cv = 2

In [16]:
from sklearn.linear_model import RidgeClassifier

try:
    with open('lc.pkl', 'rb') as f:
        src = pickle.load(f)
except FileNotFoundError:
    lin_cls = RidgeClassifier()

    # test_model(lin_cls,X,y)
    param = {"alpha":[0.001,0.1,1,2,3,5,10,100,1000]}
    src = gs_model(lin_cls, param,X,y,cv=search_cv)
    with open('lc.pkl', 'wb') as f:
        pickle.dump(src, f)
    del lin_cls

report(src)
# model = src.best_estimator_
# cv_model(model,X,y)

# dill.dump_session('hw_lc.db')

Best param: {'alpha': 100}
Best score: 0.6086975040463413


In [17]:
from sklearn.tree import DecisionTreeClassifier

try:
    with open('tc.pkl', 'rb') as f:
        src = pickle.load(f)
except FileNotFoundError:
    tree_cls = DecisionTreeClassifier()

    # test_model(tree_cls,X,y)
    param = {"max_depth":[None,2,5,10,15],
            "min_samples_split":[2,5,10,20,50,100]}
    src = gs_model(tree_cls, param,X,y,cv=search_cv)
    with open('tc.pkl', 'wb') as f:
        pickle.dump(src, f)
    del tree_cls
    
report(src)
# model = src.best_estimator_
# cv_model(model,X,y)

# dill.dump_session('hw_tc.db')

Best param: {'min_samples_split': 10, 'max_depth': 5}
Best score: 0.6055456171735242


In [18]:
import xgboost as xgb

try:
    with open('xc1.pkl', 'rb') as f:
        src = pickle.load(f)
except FileNotFoundError:
    xgb_cls = xgb.XGBClassifier(nthread=4, n_estimators=20)
    # test_model(xgb_reg,vec,df)
    param = {
    #         "n_estimators":[50,100,150,200],
            "max_depth":[2,3,5,10,15],
            "learning_rate":[0.1,1,10],
             "reg_lambda":[0.1,1,10]
            }
    src = gs_model(xgb_cls,  param,X,y,jobs=1,cv=search_cv)
    with open('xc1.pkl', 'wb') as f:
        pickle.dump(src, f)
    del xgb_cls
    
report(src)

try:
    with open('xc2.pkl', 'rb') as f:
        src = pickle.load(f)
except FileNotFoundError:
    model = src.best_estimator_
    param = {"n_estimators":[50,100,150,200]}
    src = gs_model(model,  param,X,y,jobs=1,cv=search_cv)
    with open('xc2.pkl', 'wb') as f:
        pickle.dump(src, f)

report(src)
# model = src.best_estimator_
# cv_model(model,X,y)

# dill.dump_session('hw_xc.db')

Best param: {'reg_lambda': 10, 'max_depth': 15, 'learning_rate': 0.1}
Best score: 0.5986029474401567
Best param: {'n_estimators': 100}
Best score: 0.6076752704659681


В целом предсказание работает, модели работают в целом одинаково. Значение в районе 60% определяется скорее всего из-за шума. Для улучшения предсказания, можно почистить данные и удалить тех, кто не в России или в некрупных городах.

# Проверка на тестовой выборке

In [74]:
# dill.load_session('hw_lc.db')
# with open('xy_train.pkl', 'rb') as f:
#     X = pickle.load(f)
#     y = pickle.load(f)
#     get_xy = pickle.load(f)

try:
    with open('xy_test.pkl', 'rb') as f:
        X = pickle.load(f)
        y = pickle.load(f)
#     with open('xy_train.pkl', 'rb') as f:
#         X_train = pickle.load(f)
#         y_train = pickle.load(f)
except FileNotFoundError:
    df_xy = df_test.dropna(subset=['city', 'groups'])
    
    # новые города, которых нет в обучающей выборке
    cls = get_xy.le.classes_.tolist()
    [cls.append(n) for n in df_xy['city'] if n not in cls]
    get_xy.le.classes_ = np.array(cls)
    
    X,y = get_xy.transform(df_xy)
    with open('xy_test.pkl', 'wb') as f:
        pickle.dump(X, f)
        pickle.dump(y, f)
        
with open('lc.pkl', 'rb') as f:
    src = pickle.load(f)
model = src.best_estimator_
# model.fit(X_train,y_train)
print("score:",accuracy_score(model.predict(X), y))

score: 0.6029917980536313


Качество модели не отличается от обучения, модель не переобучилась и относительно устойчива к шуму.